In [ ]:
import heapq  # Importing heapq for priority queue (min-heap) operations
import numpy as np  # Importing numpy for efficient matrix operations

class Puzzle:
    def __init__(self, board, goal):
        """
        Initializes the puzzle with the given board and goal state.

        Args:
        - board: 2D list representing the current state of the puzzle
        - goal: 2D list representing the goal state of the puzzle
        """
        self.board = np.array(board)  # Convert board to a numpy array
        self.goal = np.array(goal)  # Convert goal state to a numpy array
        self.size = len(board)  # Size of the puzzle (e.g., 3 for a 3x3 puzzle)
        self.blank_pos = tuple(np.argwhere(self.board == 0)[0])  # Find the position of the blank tile (0)

    def get_possible_moves(self):
        """
        Returns a list of possible moves by swapping the blank space with adjacent tiles.

        Returns:
        - List of tuples where each tuple contains (move direction, new puzzle state)
        """
        x, y = self.blank_pos  # Get the blank tile position
        moves = []  # List to store possible moves
        directions = {'U': (-1, 0), 'D': (1, 0), 'L': (0, -1), 'R': (0, 1)}  # Possible move directions

        for move, (dx, dy) in directions.items():
            new_x, new_y = x + dx, y + dy  # Calculate new position of the blank tile

            if 0 <= new_x < self.size and 0 <= new_y < self.size:  # Check if the move is within bounds
                new_board = self.board.copy()  # Create a copy of the board
                # Swap the blank tile with the adjacent tile
                new_board[x, y], new_board[new_x, new_y] = new_board[new_x, new_y], new_board[x, y]
                new_puzzle = Puzzle(new_board, self.goal)  # Create a new puzzle state
                moves.append((move, new_puzzle))  # Store the move and new state

        return moves

    def is_goal(self):
        """
        Checks if the current puzzle state matches the goal state.

        Returns:
        - True if the board matches the goal, otherwise False
        """
        return np.array_equal(self.board, self.goal)

    def manhattan_distance(self):
        """
        Computes the Manhattan distance heuristic.

        Returns:
        - Total Manhattan distance (sum of tile distances from their goal positions)
        """
        distance = 0
        for num in range(1, self.size ** 2):  # Iterate over all tiles (excluding blank)
            x, y = np.where(self.board == num)  # Get current position of tile
            goal_x, goal_y = np.where(self.goal == num)  # Get goal position of tile
            # Compute Manhattan distance
            distance += abs(x.item() - goal_x.item()) + abs(y.item() - goal_y.item())
        return distance

    def __eq__(self, other):
        """
        Checks if two puzzle states are equal.
        """
        return np.array_equal(self.board, other.board)

    def __hash__(self):
        """
        Generates a unique hash value for a puzzle state to store it in sets.
        """
        return hash(self.board.tobytes())

    def __lt__(self, other):
        """
        Comparison method for priority queue (min-heap), based on Manhattan distance.
        """
        return self.manhattan_distance() < other.manhattan_distance()

    def print_board(self):
        """
        Prints the current board state in a readable format.
        """
        print("\n".join(" ".join(map(str, row)) for row in self.board))
        print("\n" + "-" * 10)

def a_star_solver(start_board, goal_board):
    """
    Solves the 8-puzzle problem using the A* search algorithm.

    Args:
    - start_board: Initial board state as a 2D list
    - goal_board: Goal board state as a 2D list

    Returns:
    - List of moves to reach the goal state or None if unsolvable
    """
    start_puzzle = Puzzle(start_board, goal_board)  # Create the initial puzzle state
    pq = []  # Priority queue (min-heap)
    heapq.heappush(pq, (0, 0, start_puzzle, []))  # Push initial state with priority 0
    visited = set()  # Set to track visited states

    while pq:
        _, cost, current_puzzle, path = heapq.heappop(pq)  # Get the puzzle with the lowest cost

        current_puzzle.print_board()  # Print current board state

        if current_puzzle.is_goal():  # Check if the goal state is reached
            return path  # Return the path of moves

        visited.add(current_puzzle)  # Mark the current state as visited

        for move, next_puzzle in current_puzzle.get_possible_moves():  # Explore possible moves
            if next_puzzle not in visited:
                new_cost = cost + 1  # Increment cost
                priority = new_cost + next_puzzle.manhattan_distance()  # Calculate A* priority (f = g + h)
                heapq.heappush(pq, (priority, new_cost, next_puzzle, path + [move]))  # Add to priority queue

    return None  # Return None if no solution is found

# Example Usage
start = [[1, 2, 3], [4, 0, 5], [6, 7, 8]]  # Initial board state
goal = [[1, 2, 3], [4, 5, 6], [7, 8, 0]]  # Goal board state
solution = a_star_solver(start, goal)  # Solve the puzzle
print("Solution Path:", solution)  # Print the solution path

1 2 3
4 0 5
6 7 8

----------
1 2 3
4 5 0
6 7 8

----------
1 2 3
0 4 5
6 7 8

----------
1 2 3
4 7 5
6 0 8

----------
1 0 3
4 2 5
6 7 8

----------
1 2 3
4 5 8
6 7 0

----------
1 2 3
4 7 5
0 6 8

----------
1 2 3
4 7 5
6 8 0

----------
1 2 0
4 5 3
6 7 8

----------
1 2 3
6 4 5
0 7 8

----------
1 2 3
6 4 5
7 0 8

----------
1 2 3
6 4 5
7 8 0

----------
1 3 0
4 2 5
6 7 8

----------
0 1 3
4 2 5
6 7 8

----------
0 2 3
1 4 5
6 7 8

----------
1 2 3
4 5 8
6 0 7

----------
1 2 3
0 7 5
4 6 8

----------
1 0 2
4 5 3
6 7 8

----------
1 2 3
4 7 0
6 8 5

----------
1 2 3
4 5 8
0 6 7

----------
1 2 3
7 0 5
4 6 8

----------
1 2 3
6 0 5
7 4 8

----------
1 2 3
7 6 5
4 0 8

----------
1 2 3
7 5 0
4 6 8

----------
1 2 3
6 4 0
7 8 5

----------
1 2 3
0 6 5
7 4 8

----------
1 2 3
6 5 0
7 4 8

----------
1 2 3
7 6 5
4 8 0

----------
1 3 5
4 2 0
6 7 8

----------
4 1 3
0 2 5
6 7 8

----------
2 0 3
1 4 5
6 7 8

----------
1 2 3
4 0 8
6 5 7

----------
1 2 0
4 7 3
6 8 5

----------
1 5 2
4 0 